In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import catboost as cb
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

X_train = pd.read_csv("train/X_train.csv")
y_train = pd.read_csv("train/y_train.csv")

X_val = pd.read_csv("validation/X_val.csv")
y_val = pd.read_csv("validation/y_val.csv")

X_test = pd.read_csv("test/X_test.csv")
y_test = pd.read_csv("test/y_test.csv")

In [ ]:
# Plot data
X_train


### Regression

In [ ]:
degrees = [1, 2, 3, 4]
validation_errors = []
best_degree=1

"""for degree in degrees:
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train)
    X_val_poly = poly.transform(X_val)
    
    # Train a linear regression model on the polynomial features
    model = LinearRegression()
    model.fit(X_train_poly, y_train)
    
    # Predict on the validation set and calculate error
    y_val_pred = model.predict(X_val_poly)
    val_mse = mean_squared_error(y_val, y_val_pred)
    validation_errors.append(val_mse)
    
    print(f"Degree: {degree}, Validation MSE: {val_mse}")

# Select the best polynomial degree based on the lowest validation error
best_degree = degrees[np.argmin(validation_errors)]
print(f"Best Polynomial Degree: {best_degree}")"""

# Combine training and validation sets
X_train_val = pd.concat([X_train, X_val])
y_train_val = pd.concat([y_train, y_val])

# Create polynomial features with the best degree
poly = PolynomialFeatures(degree=best_degree)
X_train_val_poly = poly.fit_transform(X_train_val)
X_test_poly = poly.transform(X_test)

# Train the model on the combined training and validation set
final_model = LinearRegression()
final_model.fit(X_train_val_poly, y_train_val)

# Predict on the test set
y_test_pred = final_model.predict(X_test_poly)

# Calculate the Mean Squared Error
test_mse = mean_squared_error(y_test, y_test_pred)
print(f"Test Mean Squared Error: {test_mse}")

# Plot actual vs predicted values for the test set
plt.plot(range(len(y_test)), y_test, label="Actual Values")
plt.plot(range(len(y_test_pred)), y_test_pred, label="Predicted Values")
plt.legend()
plt.xlabel("Time Steps (Test Data)")
plt.ylabel("Value")
plt.show()


### Neural Network

In [ ]:
#neural network
model = MLPRegressor(hidden_layer_sizes=(10,), max_iter=2000, random_state=0)

# Train the model on the training data
model.fit(X_train, y_train)
# Predict on the test set
y_pred = model.predict(X_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Plot actual vs predicted values for the test set
plt.plot(range(len(y_test)), y_test, label="Actual Values")
plt.plot(range(len(y_pred)), y_pred, label="Predicted Values")
plt.legend()
plt.xlabel("Time Steps (Test Data)")
plt.ylabel("Value")
plt.show()

### Decision Tree

In [ ]:
"""
catboost_model = cb.CatBoostRegressor(verbose=0, random_state=42)
catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val))

lightgbm_model = lgb.LGBMRegressor(verbose=-1, random_state=42)
lightgbm_model.fit(X_train, y_train, eval_set=(X_val, y_val))

xgboost_model = xgb.XGBRegressor(random_state=42)
xgboost_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

y_pred_catboost = catboost_model.predict(X_test)
mse_catboost = mean_squared_error(y_test, y_pred_catboost)
print(f"Mean Squared Error Catboost: {mse_catboost}")

y_pred_lightgbm = lightgbm_model.predict(X_test)
mse_lightgbm = mean_squared_error(y_test, y_pred_lightgbm)
print(f"Mean Squared Error LightGBM: {mse_lightgbm}")

y_pred_xgboost = xgboost_model.predict(X_test)
mse_xgboost = mean_squared_error(y_test, y_pred_xgboost)
print(f"Mean Squared Error XGBoost: {mse_xgboost}")

y_pres_ensamble = (y_pred_catboost + y_pred_lightgbm + y_pred_xgboost) / 3
mse_ensamble = mean_squared_error(y_test, y_pres_ensamble)
print(f"Mean Squared Error Ensamble: {mse_ensamble}")

plt.plot(range(len(y_test)), y_test, label="Actual Values")
plt.plot(range(len(y_pred)), y_pred, label="Predicted Values")
plt.legend()
plt.title("Gradient Boosting decision tree (Ensamble of 3 models)")
plt.xlabel("Time Steps (Test Data)")
plt.ylabel("Value")
plt.show()
"""

# Decision tree regressor
model_dt = DecisionTreeRegressor(random_state=42)
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)
mse_dt = mean_squared_error(y_test, y_pred_dt)
print(f"Mean Squared Error Decision Tree: {mse_dt}")

plt.plot(range(len(y_test)), y_test, label="Actual Values")
plt.plot(range(len(y_pred_dt)), y_pred_dt, label="Predicted Values")
plt.legend()
plt.title("Decision Tree Regressor")
plt.xlabel("Time Steps (Test Data)")
plt.ylabel("Value")
plt.show()

### Random forrest

In [ ]:
model_rf = RandomForestRegressor(random_state=42)
model_rf.fit(X_train, y_train, )
y_pred_rf = model_rf.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Mean Squared Error Random Forest: {mse_rf}")

plt.plot(range(len(y_test)), y_test, label="Actual Values")
plt.plot(range(len(y_pred_rf)), y_pred_rf, label="Predicted Values")
plt.legend()
plt.title("Random Forest")
plt.xlabel("Time Steps (Test Data)")
plt.ylabel("Value")
plt.show()

### SVM with kernel

In [ ]:
# SVM regressor (Support Vector Machine)
from sklearn.svm import SVR
model_svm = SVR()
model_svm.fit(X_train, y_train)
y_pred_svm = model_svm.predict(X_test)
mse_svm = mean_squared_error(y_test, y_pred_svm)
print(f"Mean Squared Error SVM: {mse_svm}")

plt.plot(range(len(y_test)), y_test, label="Actual Values")
plt.plot(range(len(y_pred_svm)), y_pred_svm, label="Predicted Values")
plt.legend()
plt.title("Support Vector Machine")
plt.xlabel("Time Steps (Test Data)")
plt.ylabel("Value")
plt.show()